# Initial Setup

In [1]:
import os
import re
import gradio as gr
import ollama
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
!ollama pull llama3.2

In [ ]:
!ollama pull deepseek-r1:latest

In [4]:
# Online Bot
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "type-your-api-key-here")
GOOLE_API_KEY = os.getenv("GOOGLE_API_KEY", "type-your-api-key-here")
GOOGLE_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gpt_api = OpenAI()
gemini_api = OpenAI(api_key=GOOLE_API_KEY, base_url=GOOGLE_BASE_URL)

gpt_model = "gpt-4o-mini"
gemini_model = "gemini-2.0-flash"

In [5]:
# Local Bot
OLLAMA_API_KEY = "ollama"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

llama_api = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)
deepseek_api = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)

llama_model = "llama3.2"
deepseek_model = "deepseek-r1:latest"

# Prompting

In [6]:
system_prompt = "You are an assistant with excellent writing skills. \
You are able to paraphrase text according to user requests. \
Respond in markdown."

def generate_user_prompt(text: str, tone: str) -> str:
    user_prompt = f"Paraphrase this text to make it more {tone.lower()}.\n"
    user_prompt += f"{text}\n"
    user_prompt += f"Just paraphrase the text without adding any comments or feedbacks."
    return user_prompt

def generate_messages(text: str, tone: str, system_prompt: str=system_prompt) -> list:
    return [
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":generate_user_prompt(text, tone)}
    ]

def set_bot(model_name: str) -> tuple:
    if model_name == "Llama":
        api = llama_api
        model = llama_model
    elif model_name == "Deep Seek":
        api = deepseek_api
        model = deepseek_model
    elif model_name == "GPT":
        api = gpt_api
        model = gpt_model
    elif model_name == "Gemini":
        api = gemini_api
        model = gemini_model
    else:
        raise ValueError(f"Unknown model: {model_name}")
    return api, model

def extract_final_paraphrase(text: str) -> str:
    """Remove non-markdown elements"""
    match = re.search(r'</think>\s*\n*(.*)', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def stream_paraphrase(messages: str, tone: str, model_name: str):
    api, model = set_bot(model_name)
    messages = generate_messages(messages, tone)

    if model_name != "Deep Seek":
        stream = api.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
        result = ""
        for chunk in stream:
            result += chunk.choices[0].delta.content or ""
            yield result
    else:
        completion = api.chat.completions.create(
            model=model,
            messages=messages
        )
        result = completion.choices[0].message.content
        result = extract_final_paraphrase(result)
        yield result

# UI

In [7]:
view = gr.Interface(
    fn=stream_paraphrase,
    inputs=[
        gr.Textbox(label="Input your text", lines=8),
        gr.Dropdown(choices=["Professional", "Academic", "Casual"], label="Select tone", value="Professional"),
        gr.Dropdown(choices=["Llama", "Deep Seek", "GPT", "Gemini"], label="Select model", value="Llama")
    ],
    outputs=[gr.Markdown()],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
